Task = Data Acquisition

Author - Tejas Lohia

Date - 23/10/2024

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os

Code to acquire all the companies listed on Nasdaq.

In [14]:
api_key = "LC2DGDLSUALZRVAN"
url = f"https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={api_key}"
response = requests.get(url)

if response.status_code == 200:
    data = pd.read_csv(io.StringIO(response.text))
    nasdaq_stocks = data[data['exchange'] == 'NASDAQ']

print(nasdaq_stocks.head())
print(nasdaq_stocks.shape)

   symbol                                  name exchange assetType  \
4    AACG                 ATA Creativity Global   NASDAQ     Stock   
8    AADI                   Aadi Bioscience Inc   NASDAQ     Stock   
9    AADR  ADVISORSHARES DORSEY WRIGHT ADR ETF    NASDAQ       ETF   
10    AAL           American Airlines Group Inc   NASDAQ     Stock   
14   AAME                Atlantic American Corp   NASDAQ     Stock   

       ipoDate  delistingDate  status  
4   2008-01-29            NaN  Active  
8   2017-08-08            NaN  Active  
9   2010-07-21            NaN  Active  
10  2005-09-27            NaN  Active  
14  1984-09-07            NaN  Active  
(5227, 7)


In [17]:
#making a csv file of all the nasdaq stocks companies only with symbol, name, exchange asnd asset type

final_list = nasdaq_stocks[['symbol', 'name', 'exchange', 'assetType']]

final_list.to_csv('nasdaq_stocks.csv', index=False)

Now writing a code to acquire the historical data of the companies listed on Nasdaq.
For each of the company, acquiring the historical data, and converting into a csv file.

In [18]:

def historical_data(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = pd.read_csv(io.StringIO(response.text))
        data = data.rename(columns={'timestamp': 'date'})
        data['date'] = pd.to_datetime(data['date'])
        data.set_index('date', inplace=True)
        data = data.sort_index()
        return data
        

In [29]:

symbol_list = final_list['symbol'].tolist()

In [30]:
symbol_list[:10]

['AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPL']

In [34]:
def get_historical_data(symbol, function='TIME_SERIES_DAILY', output_size='full'):
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": "LC2DGDLSUALZRVAN",
        "outputsize": output_size,
        "datatype": "json"
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        time_series = data.get('Time Series (Daily)', {})  # Adjust according to function
        if time_series:
            df = pd.DataFrame.from_dict(time_series, orient='index')
            df = df.rename(columns={
                '1. open': 'Open',
                '2. high': 'High',
                '3. low': 'Low',
                '4. close': 'Close',
                '5. volume': 'Volume'
            })
            df.index = pd.to_datetime(df.index)
            return df.sort_index()
        else:
            print("No data found.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

# Example usage:
symbol = symbol_list[0]
data = get_historical_data(symbol)
if data is not None:
    print(data[:10])  # Show the first few rows
print(data.shape)

              Open     High     Low   Close   Volume
2008-01-29  9.5000   9.9900  8.5700  8.7500  1489000
2008-01-30  8.7500   9.1500  8.3000  8.5000   219000
2008-01-31  8.4900  10.3000  8.4900  9.5500   182300
2008-02-01  9.9300   9.9400  9.5000  9.5100    28200
2008-02-04  9.5000   9.7120  9.5000  9.5000     8300
2008-02-05  9.2600   9.5000  9.0300  9.1100    29500
2008-02-06  9.0000   9.4300  8.6700  9.0000    16000
2008-02-07  9.0000   9.5000  9.0000  9.4000     6000
2008-02-08  9.1200   9.2900  9.0000  9.0100     7900
2008-02-11  9.0500   9.1700  9.0000  9.0400     3500
(4210, 5)


In [35]:
import os
import requests
import pandas as pd

def get_historical_data(symbol, function='TIME_SERIES_DAILY', output_size='full'):
    url = "https://www.alphavantage.co/query"
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": "LC2DGDLSUALZRVAN",  # Replace with your actual API key
        "outputsize": output_size,
        "datatype": "json"
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        time_series = data.get('Time Series (Daily)', {})  # Adjust according to function
        if time_series:
            df = pd.DataFrame.from_dict(time_series, orient='index')
            df = df.rename(columns={
                '1. open': 'Open',
                '2. high': 'High',
                '3. low': 'Low',
                '4. close': 'Close',
                '5. volume': 'Volume'
            })
            df.index = pd.to_datetime(df.index)
            return df.sort_index()
        else:
            print(f"No data found for {symbol}.")
            return None
    else:
        print(f"Error fetching data for {symbol}: {response.status_code}")
        return None

def save_data_to_csv(symbol_list):
    # Create a folder named 'Hist_data' if it doesn't exist
    folder_name = 'Hist_data'
    os.makedirs(folder_name, exist_ok=True)

    for symbol in symbol_list:
        print(f"Fetching data for {symbol}...")
        data = get_historical_data(symbol)
        if data is not None:
            file_path = os.path.join(folder_name, f"{symbol}.csv")
            data.to_csv(file_path)
            print(f"Saved {symbol} data to {file_path}.")
        else:
            print(f"Skipping {symbol} due to missing data.")

# Example usage:
  # Replace with your symbols
save_data_to_csv(symbol_list[:10])

Fetching data for AACG...
Saved AACG data to Hist_data/AACG.csv.
Fetching data for AADI...
Saved AADI data to Hist_data/AADI.csv.
Fetching data for AADR...
Saved AADR data to Hist_data/AADR.csv.
Fetching data for AAL...
Saved AAL data to Hist_data/AAL.csv.
Fetching data for AAME...
Saved AAME data to Hist_data/AAME.csv.
Fetching data for AAOI...
Saved AAOI data to Hist_data/AAOI.csv.
Fetching data for AAON...
Saved AAON data to Hist_data/AAON.csv.
Fetching data for AAPB...
Saved AAPB data to Hist_data/AAPB.csv.
Fetching data for AAPD...
Saved AAPD data to Hist_data/AAPD.csv.
Fetching data for AAPL...
Saved AAPL data to Hist_data/AAPL.csv.
